In [1]:
import numpy as np
import torch
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader


In [2]:
def preprocess_data(csv_file, sequence_length):
    
  
    df = pd.read_csv(csv_file)
    
    features = df[['magnitude', 'cdi', 'mmi', 'dmin', 'gap', 'depth']].values
    
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(features)
    
    sequences = []
    for i in range(0, len(normalized_features) - sequence_length + 1, sequence_length // 2):  
        sequence = normalized_features[i:i + sequence_length]
        sequences.append(sequence)
    
    return torch.FloatTensor(sequences), scaler

In [3]:
normal_sequences, normal_scaler = preprocess_data("Normal.csv", 50)
anomalous_sequences, anomalous_scaler = preprocess_data("Anomalous.csv", 50)


/tmp/ipykernel_7607/2731436145.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400268359/work/torch/csrc/utils/tensor_new.cpp:261.)
  return torch.FloatTensor(sequences), scaler


In [4]:

from backup import TsunamiDetector


In [5]:
# Initialize the model
sequence_length = 50  # Sequence length for training (you already have it)
latent_dim = 100  # Latent dimension for the generator (can be adjusted)
lambda_ano = 0.1 # Weight for anomaly loss
num_features = 6 # Number of features (as in the input data)


# Create the TsunamiDetector instance
tsunami_detector = TsunamiDetector(sequence_length, latent_dim, lambda_ano, num_features,device='cpu')

# Train the model on the normal sequences (normal_sequences is already preprocessed)
batch_size = 64
epochs = 200

# Convert normal_sequences to a DataLoader compatible format (TensorDataset)
normal_data = TensorDataset(normal_sequences)
normal_dataloader = DataLoader(normal_data, batch_size=batch_size, shuffle=True)

# Train the model
tsunami_detector.train(normal_dataloader, epochs=epochs)



Epoch 0: G_loss: 0.8254, D_loss: 0.7625
Epoch 1: G_loss: 0.6533, D_loss: 0.8241
Epoch 2: G_loss: 0.5385, D_loss: 0.8866
Epoch 3: G_loss: 0.4829, D_loss: 0.9297
Epoch 4: G_loss: 0.4896, D_loss: 0.9146
Epoch 5: G_loss: 0.5020, D_loss: 0.8467
Epoch 6: G_loss: 0.4979, D_loss: 0.8793
Epoch 7: G_loss: 0.5094, D_loss: 0.8904
Epoch 8: G_loss: 0.5170, D_loss: 0.8343
Epoch 9: G_loss: 0.5252, D_loss: 0.8283
Epoch 10: G_loss: 0.5470, D_loss: 0.7913
Epoch 11: G_loss: 0.5500, D_loss: 0.8225
Epoch 12: G_loss: 0.5676, D_loss: 0.7975
Epoch 13: G_loss: 0.5785, D_loss: 0.7650
Epoch 14: G_loss: 0.5729, D_loss: 0.7484
Epoch 15: G_loss: 0.6323, D_loss: 0.7186
Epoch 16: G_loss: 0.6616, D_loss: 0.6979
Epoch 17: G_loss: 0.6967, D_loss: 0.6696
Epoch 18: G_loss: 0.7265, D_loss: 0.6728
Epoch 19: G_loss: 0.6942, D_loss: 0.6548
Epoch 20: G_loss: 0.7413, D_loss: 0.6293
Epoch 21: G_loss: 0.7527, D_loss: 0.6287
Epoch 22: G_loss: 0.7983, D_loss: 0.6431
Epoch 23: G_loss: 0.8037, D_loss: 0.6172
Epoch 24: G_loss: 0.8409, 

In [6]:
# Calibrate threshold with normal data
validation_data = normal_sequences  # You can use a validation set or a portion of normal data
threshold_percentile = 95
threshold = tsunami_detector.calibrate_threshold(validation_data, percentile=threshold_percentile)

print(f"Calibrated threshold: {threshold}")


Calibrated threshold: 3.698414981365204


In [7]:
# Assuming anomalous_sequences is a tensor of anomalous data (like actual tsunami events)
anomalous_sequences = torch.FloatTensor(anomalous_sequences)  # Ensure it's a tensor if not already

# Evaluate anomalous data
for sequence in anomalous_sequences:
    result = tsunami_detector.detect_tsunami(sequence)
    print(f"ANOMALY: Is Tsunami: {result['is_tsunami']}, Confidence: {result['confidence']}, Anomaly Score: {result['anomaly_score']}")


ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.151632070541382
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.23787784576416
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.1384670734405518
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.7387137413024902
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.7849591970443726
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.1453475952148438
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.0367000102996826
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.776874303817749
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.6826930046081543
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.5733755826950073
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.6298320293426514
ANOMALY: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.876802921295166


In [8]:
normal_sequences = torch.FloatTensor(normal_sequences)  # Ensure it's a tensor if not already

# Evaluate anomalous data
for sequence in normal_sequences:
    result = tsunami_detector.detect_tsunami(sequence)
    print(f"NORMAL: Is Tsunami: {result['is_tsunami']}, Confidence: {result['confidence']}, Anomaly Score: {result['anomaly_score']}")


NORMAL: Is Tsunami: True, Confidence: 1.0, Anomaly Score: 3.9357402324676514
NORMAL: Is Tsunami: True, Confidence: 1.0, Anomaly Score: 4.121284484863281
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 3.0408267974853516
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.696479082107544
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.681331157684326
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.6437221765518188
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.7291710376739502
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.9773049354553223
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.3659615516662598
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 2.065526008605957
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.5512144565582275
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.4628604650497437
NORMAL: Is Tsunami: False, Confidence: 1.0, Anomaly Score: 1.885095834